In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pandas as pd

: 

In [ ]:
spark = SparkSession.builder.master("local[*]") \
                    .appName('MovieRecomender') \
                    .getOrCreate()
print(spark.sparkContext)
print("Spark App Name : "+ spark.sparkContext.appName)

In [ ]:
import os 
os.chdir ('/home/jovyan/work/MovieRecomender')
#change to your working directory
%pwd

In [ ]:
df_m = pd.read_csv("data/movies.dat", engine='python', sep='::', names=["MovieID", "Title", "Genres"],encoding='ISO-8859-1')
df_m = spark.createDataFrame(df_m)

df_r = pd.read_csv("data/ratings.dat", engine='python', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"])
df_r = spark.createDataFrame(df_r)

df_u = pd.read_csv("data/users.dat", engine='python', sep='::', names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])
df_u = spark.createDataFrame(df_u)

In [ ]:
df_m.printSchema()
df_m.show(3)

In [ ]:
df_r.printSchema()
df_r.show(3)

In [ ]:
df_u.printSchema()
df_u.show(3)

In [ ]:
df=df_m.join(df_r,"MovieID","inner").join(df_u,"UserID","inner")
df.printSchema()
df.show(3)

In [ ]:
df=df.select(["UserID","MovieID","Rating"])
df.na.drop()
df.show(3)

In [ ]:
(train, test) = df.randomSplit([0.8, 0.2])

In [ ]:
als = ALS(userCol="UserID", itemCol="MovieID", ratingCol="Rating", nonnegative = True, implicitPrefs = False)

In [ ]:
grid_search = ParamGridBuilder().addGrid(als.rank, [10, 25, 50 ,75 , 100]  ).addGrid(als.maxIter,[5, 10, 20, 40, 80, 160]  ).addGrid(als.regParam, [.001,.005,.01, .05, .1, .5] ).build()

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction") 

In [ ]:
cv = CrossValidator(estimator=als, estimatorParamMaps=grid_search, evaluator=evaluator, numFolds=5)

In [ ]:
cv_fitted=cv.fit(train)